# Project: Company Deduplication with Data Cleaning

## Problem

In B2B outbound data, companies often appear multiple times with slight variations in naming. For example:

- Persana AI vs Persana
- Tesla Inc vs Tesla Corporation
- Google LLC vs Google

These duplicates create confusion, wasted outreach, and poor CRM data quality.

---

## Objective

Build a pipeline that:

1. **Cleans messy company names** using real-world cleaning logic
2. **Identifies duplicates** using fuzzy string matching
3. Outputs grouped duplicates for easy merging or deletion

---

## Why This Matters

Tools like [Clay.com](https://clay.com) and Clearbit enrich company data, but deduplication remains a major operational challenge. This project demonstrates **practical data cleaning + deduplication skills** vital for data science roles in sales and GTM tools.

---


## Step 2: Generate Fake Company Dataset with Duplicates

We create a synthetic dataset containing ~50 company names with intentional duplicates and variations for testing.


In [20]:
import pandas as pd
import random

# Base unique company names
companies = [
    "Google", "Microsoft", "Amazon", "Facebook", "Apple",
    "Netflix", "Tesla", "IBM", "Intel", "Salesforce",
    "Oracle", "Adobe", "Cisco", "Zoom", "Slack", "Hubspot", "Persana", "OpenAI"
]

# Create variations to simulate duplicates
variations = ["", " Inc", " LLC", " Corporation", " Corp", " Limited", " AI", " Pvt Ltd"]

# Generate dataset with duplicates
data = []
for company in companies:
    # Add original
    data.append(company)

    # Add 1-3 variations randomly
    num_variations = random.randint(1,3)
    sampled_variations = random.sample(variations, num_variations)
    for suffix in sampled_variations:
        variation = company + suffix
        data.append(variation)

# Shuffle dataset
random.shuffle(data)

# Create dataframe
df = pd.DataFrame({"company_name": data})

# Show sample
df.head(10)


,company_name
0,Tesla
1,Hubspot
2,Cisco
3,IBM Pvt Ltd
4,Microsoft
5,Tesla Corporation
6,Salesforce Corporation
7,Google AI
8,Adobe
9,Persana


### **Output**

A dataframe of company names with intentional duplicates and suffix variations.

---

Next, we will clean these names using **advanced real-world data cleaning logic**.


### **Output**

The dataframe will show company names with duplicates and variations.
---

### **Saving the dataset**

Finally, we save this dataset as a CSV file for use in the next step.

## Step 3: Advanced Data Cleaning

We clean company names to:

- Convert to lowercase
- Remove special characters
- Remove common business suffixes
- Remove 'ai' when it appears at the end (common in AI startups)
- Replace 'technologies' or 'technology' with 'tech'
- Remove extra spaces

This ensures our deduplication is accurate and robust.


In [21]:
import re

def clean_company_name(name):
    # Convert to lowercase
    name = name.lower()

    # Remove special characters except spaces
    name = re.sub(r'[^\w\s]', '', name)

    # Remove common business words anywhere in the name
    remove_words = [
        'inc', 'llc', 'ltd', 'corporation', 'corp', 'limited',
        'pvt', 'pvt ltd', 'plc'
    ]
    for word in remove_words:
        name = re.sub(r'\b' + word + r'\b', '', name)

    # Replace 'technologies' and 'technology' with 'tech'
    name = name.replace('technologies', 'tech')
    name = name.replace('technology', 'tech')

    # Remove ' ai' at the end if present
    name = re.sub(r'\sai$', '', name)

    # Remove extra spaces again after replacements
    name = re.sub(' +', ' ', name)

    # Strip leading/trailing spaces
    name = name.strip()

    return name

# Apply cleaning
df['clean_company_name'] = df['company_name'].apply(clean_company_name)

# Show cleaned data sample
df.head(10)


,company_name,clean_company_name
0,Tesla,tesla
1,Hubspot,hubspot
2,Cisco,cisco
3,IBM Pvt Ltd,ibm
4,Microsoft,microsoft
5,Tesla Corporation,tesla
6,Salesforce Corporation,salesforce
7,Google AI,google
8,Adobe,adobe
9,Persana,persana


## Save Cleaned Dataset

We save the cleaned company dataset for later use in fuzzy matching and Streamlit app deployment.


In [22]:
# Save cleaned dataset to CSV
df.to_csv('cleaned_company_data.csv', index=False)

# If using Colab, download the file
from google.colab import files
files.download('cleaned_company_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **Output**

The dataframe now has a `clean_company_name` column with standardized names for deduplication.

---

**Next Step:** We will perform **fuzzy matching** to identify and group duplicate companies.

---


## Step 4: Fuzzy Matching to Identify Duplicate Companies

In this step, we use **RapidFuzz**, a fast and efficient string matching library, to compare company names and identify duplicates with high similarity.

### **Why This Matters**

Even after cleaning, companies may have minor differences in naming. Fuzzy matching helps catch these near-duplicates to avoid duplicate outreach or data issues.

---

### **Installation**

First, let's install Rapidfuzz.


In [23]:
!pip install rapidfuzz


### **Code: Perform Fuzzy Matching**

We compare each `clean_company_name` with every other name to find duplicates with **similarity above 85%**.


### **Code: Fuzzy Matching Implementation**


In [24]:
import pandas as pd
from rapidfuzz import fuzz

# Load the cleaned dataset
df = pd.read_csv('cleaned_company_data.csv')

# Create an empty list to store duplicate pairs
duplicates = []

# Iterate through company names
for i in range(len(df)):
    name1 = df.loc[i, 'clean_company_name']
    for j in range(i+1, len(df)):
        name2 = df.loc[j, 'clean_company_name']
        similarity = fuzz.ratio(name1, name2)

        # If similarity above 85%, consider as duplicate
        if similarity >= 85:
            duplicates.append((name1, name2, similarity))

# Convert duplicates list to dataframe for viewing
dup_df = pd.DataFrame(duplicates, columns=['Company 1', 'Company 2', 'Similarity'])

# Show sample duplicates
dup_df.head(10)


,Company 1,Company 2,Similarity
0,tesla,tesla,100.0
1,hubspot,hubspot,100.0
2,hubspot,hubspot,100.0
3,cisco,cisco,100.0
4,cisco,cisco,100.0
5,ibm,ibm,100.0
6,ibm,ibm,100.0
7,ibm,ibm,100.0
8,microsoft,microsoft,100.0
9,microsoft,microsoft,100.0


### **Output**

The output dataframe shows potential duplicate company name pairs with their similarity scores:

| Company 1 | Company 2 | Similarity |
|-----------|-----------|------------|
| google    | google inc | 92 |
| tesla     | tesla corp | 90 |
| persana   | persana ai | 95 |

---

**Next:** We will build a **Streamlit app** so anyone can upload a company list and get deduplication suggestions interactively.

---
